In [9]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import plotly.io as pio
from creds import *
import seaborn as sns
import numpy as np
px.set_mapbox_access_token(mapbox_access_token)
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import re
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from matplotlib.pylab import rcParams
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from ipywidgets import widgets
from fbprophet import Prophet

In [37]:
df_btc = pd.read_csv('BTC-USD_5Y.csv')

In [38]:
#LSTM Stock Prediction Model 
#Model from https://towardsdatascience.com/lstm-time-series-forecasting-predicting-stock-prices-using-an-lstm-model-6223e9644a2f
scaler=MinMaxScaler(feature_range=(0,1))
df_btc["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_btc.index=df_nse['Date']
data=df_btc.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])
#Sort the dataset on date time and filter “Date” and “Close” columns:
for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]
new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)
dataset=new_data.values
train=dataset[0:987,:] #Data before Jan 2019
valid=dataset[987:,:]
# Normalize the new filtered dataset:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
x_train,y_train=[],[]
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
# Load the LSTM model:
model=load_model("LSTM_model.h5")
inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)
X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)
train=new_data[:987]
valid=new_data[987:] #Predicted Price
valid['LSTM Predictions']=closing_price 

In [39]:
#reread a clean dataset 
btc = pd.read_csv("BTC-USD_5Y.csv") 

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2016-05-10,NaT,460.518005,461.928986,448.954010,450.894989,450.894989,58956100.0
2016-05-11,NaT,450.864014,454.575989,450.864014,452.727997,452.727997,50605200.0
2016-05-12,NaT,452.446991,454.949005,449.250000,454.765991,454.765991,59849300.0
2016-05-13,NaT,454.850006,457.054993,453.453003,455.670013,455.670013,60845000.0
2016-05-14,NaT,455.822998,456.835999,454.786011,455.670990,455.670990,37209000.0


In [40]:
#Facebook’s Prophet Model
#Model from https://towardsdatascience.com/time-series-forecasting-predicting-stock-prices-using-an-arima-model-2e3b3080bd70
data = btc[["Date","Close"]] # select Date and Price
# Rename the features: These names are NEEDED for the model fitting
data = data.rename(columns = {"Date":"ds","Close":"y"})
m = Prophet(daily_seasonality = True) # the Prophet class (model)
m.fit(data[:987]) #Trained data before Jan 2019
future = m.make_future_dataframe(periods=845) #we need to specify the number of days in future
prediction = m.predict(future)
prediction = prediction.rename(columns = {"ds":"Date",'yhat_upper':"Prophet Prediction"})
prediction.head()

,Date,trend,yhat_lower,Prophet Prediction,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-05-10,582.572792,-345.306829,2137.485035,582.572792,582.572792,321.804362,321.804362,321.804362,557.492353,...,-1.751620,-1.751620,-1.751620,-233.936370,-233.936370,-233.936370,0.0,0.0,0.0,904.377155
1,2016-05-11,576.726901,-386.652041,2043.832965,576.726901,576.726901,278.363642,278.363642,278.363642,557.492353,...,-18.644004,-18.644004,-18.644004,-260.484706,-260.484706,-260.484706,0.0,0.0,0.0,855.090544
2,2016-05-12,570.881010,-348.252296,2060.271392,570.881010,570.881010,257.341719,257.341719,257.341719,557.492353,...,-8.387991,-8.387991,-8.387991,-291.762642,-291.762642,-291.762642,0.0,0.0,0.0,828.222729
3,2016-05-13,565.035119,-408.784740,1941.513792,565.035119,565.035119,224.200449,224.200449,224.200449,557.492353,...,-6.395349,-6.395349,-6.395349,-326.896555,-326.896555,-326.896555,0.0,0.0,0.0,789.235568
4,2016-05-14,559.189228,-415.696242,1979.095989,559.189228,559.189228,213.157830,213.157830,213.157830,557.492353,...,20.597092,20.597092,20.597092,-364.931615,-364.931615,-364.931615,0.0,0.0,0.0,772.347058


In [62]:
#Merge two pediction model with real bitcoin daily close price
Prophet_prediction = prediction[['Date','Prophet Prediction']]
Prophet_prediction = Prophet_prediction[987:] 
result = pd.merge(valid, df_nse, how="outer", on=["Close"])
result = pd.merge(result, Prophet_prediction, how="outer", on=["Date"])
result.drop(columns=['Open','High','Low','Adj Close','Volume'])
result = result.reindex(columns = ['Date','Close','LSTM Predictions','Prophet Prediction'])#reorder columns

In [49]:
#Visulization of real Bitcoin close price, Prophet model, and LSTM model
fig = go.Figure([go.Scatter(x=result['Date'], y=result['Close'],name = "Bitcoin Trends")])
fig.add_trace(go.Scatter(x=result['Date'], y=result['Prophet Prediction'],mode='lines',name='Prophet Prediction'))
fig.add_trace(go.Scatter(x=result['Date'], y=result['LSTM Predictions'],mode='lines',name='LSTM Prediction'))
fig.update_layout(
    title='Bitcoin Prediction Model',
    yaxis_title='Bitcoin - USD',
    shapes = [dict(
        x0='2020-04-10', x1='2020-12-13', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2020-04-10', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='LSTM Missing Data')] #Missing part of LSTM model, I also don't know the reason
)
fig.show()
pio.write_html(fig,"Bitcoin Pediction Model.html", full_html=False)

In [50]:
#Table of predicted price and real price
fig = go.Figure(data=[go.Table(header=dict(values=list(result.columns),fill_color='paleturquoise',align='left'),
                cells=dict(values=[result['Date'],result['Close'],result['LSTM Predictions'],result['Prophet Prediction']],fill_color='lavender',align ='left'))
])  

fig.show()
pio.write_html(fig,"Prediction Model Table.html", full_html=False)

In [64]:
prediction_corr = result.corr(method ='pearson')
prediction_corr.head()

,Close,LSTM Predictions,Prophet Prediction
Close,1.000000,0.994402,0.795411
LSTM Predictions,0.994402,1.000000,0.841196
Prophet Prediction,0.795411,0.841196,1.000000
